In [ ]:
# https://github.com/ecmwf/ecmwf-opendata

# INSTALING DEPENDENCIES

In [ ]:
#!pip3 install ipyleaflet
#!pip3 install ipywidgets

# IMPORT LIBRARIES

In [1]:
# WIDGETS
from ipywidgets import interact
import ipywidgets as widgets

# DATA
from ecmwf.opendata import Client
from eccodes import *
import numpy as np
import xarray as xr

# manage data
import pdbufr # manage bufr data
from datetime import datetime, timedelta # date functions
import pandas as pd # manage dataframes
from Magics.macro import * # load grib data

# plot mapa
import ipyleaflet



# FUNCTIONS

In [2]:
def print_date_range(date_range):
    start, end = date_range
    s = start.strftime('%d %b %Y %H:%M')
    e = end.strftime('%d %b %Y %H:%M')
    print(f'Temporal horizon: {s} - {e}')
    
def update_horizon_ens(change):
    options = [(timestep.strftime('%d/%m %H'), timestep) for timestep in tracks_date[change.new - 1]]
    temporal_horizon_slider.options = options
    temporal_horizon_slider.index = (0, len(options)-1)
# Function to convert datetime to integer
def datetime_to_int(dt):
    return int((dt - start_dte).total_seconds() // step.total_seconds())

# Function to convert interger to datetime
def int_to_datetime(ts):
    return start_dte + ts * step

def print_timestep(dt):
    date = int_to_datetime(dt)
    print('Selected date: ', date.strftime('%d %b %Y %H:%M'))
    
# Link the timestep selection slider to the horizon slider
def update_snapshot_slider(change):
    start, end = change.new
    timestep_slider.value = datetime_to_int(start.to_pydatetime())
    timestep_slider.min = datetime_to_int(start.to_pydatetime())
    timestep_slider.max = datetime_to_int(end.to_pydatetime())
# Link the cyclone track to the temporal horizon slider
def update_track_location(change):
    start, end = change.new
    start_index = datetime_to_int(start.to_pydatetime())
    end_index = datetime_to_int(end.to_pydatetime())
    track.locations = locations[member.value - 1][start_index:end_index+1]
def update_marker_location(change):
    marker.location = locations[member.value - 1][change.new]


# PARAMETERS (will be read from main widget)

In [96]:
time = 0
date = 20230806
step = 12

initial_lat_lon = (38, -90)

#########################################
#    date=0, # today
#    date=-1, # yesterday
#    date=-2, # the day before yesterday

# other parameters

In [125]:
outfolder = ''


# variable initialization

In [126]:
lat = initial_lat_lon[0]
lon = initial_lat_lon[1]
atm_variables = ['2t', 'msl']

# DOWNLOAD DATA

In [102]:
for atm in atm_variables:
    outname = outfolder + atm +  '.grib'
    client = Client("https://data.ecmwf.int/forecasts", beta=True) # ecwf: last five days
    client.retrieve(
        date = date, #date start of the forecast
        time = time,  # time start of the forecast or 12
        step = 240, #step of the forecast
        stream = "oper",
        type = "fc",
        levtype = "sfc",
        param = atm,
        target = outname
)

20230806000000-240h-oper-fc.grib2:   0%|          | 0.00/281k [00:00<?, ?B/s]

20230806000000-240h-oper-fc.grib2:   0%|          | 0.00/234k [00:00<?, ?B/s]

# LOAD DATA INTO A DATAFRAME

In [103]:
df = pd.DataFrame()
flag = 1
for atm in atm_variables:
    with xr.open_dataset(outname, engine="cfgrib") as data:
        ds = data
    if flag == 1:
        df.assign(Latitude=ds.latitude.values, Longitude=ds.longitude.values)
        flag = 0
    locations = []
    new_column = []
    for ii in range(0,len(ds.latitude),1):
        for jj in range(0,len(ds.longitude),1):
            # here I don't know how to get the variabe value from an index instad than
            # using the variable name so it  can be used for all the variables in the loop
            new_value = ds.t2m.values[ii,jj]
            new_column.append(new_value)
            #locs = [ds.latitude.values[ii], ds.longitude.values[jj], new_value]
            #locations.append(locs)
    

# CREATE MAP WITHOUT LAYERS

In [134]:
map_canvas = ipyleaflet.Map(
    center= initial_lat_lon,
    basemap=ipyleaflet.basemaps.OpenStreetMap.France,
    zoom = 3.5,
)


# SELECT VARIABLE TO PLOT

In [ ]:
varid = widgets.Dropdown(
    options = ['t2m'],
    description = 'Atmospheric variable:',
    disabled=False,
)

# CREATE LAYER 

In [97]:
# test for a specific variable
layer_mslp = Heatmap(
    locations=locations_mslp,
    radius=5
)

map_canvas.add_layer(layer_mslp)

In [139]:
# test for a specific variable
layer_t2m =  Heatmap(locations=locations, radius=10)
map_canvas.add_layer(layer_t2m)

In [129]:
# here the variable name (varid.value) will be read from the widget
locations = df['Latitude', 'Longitude', varid.value]
new_layer  =  Heatmap(locations=locations, radius=7, blur=10)
map_canvas.add_layer(new_layer)

KeyError: ('Latitude', 'Longitude', 't2m')

In [138]:
map_canvas

Map(bottom=1780.0, center=[38, -90], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…